In [1]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Activation, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras import backend
from tensorflow.keras.preprocessing.image import img_to_array, ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import DenseNet201
from imutils import paths
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os

In [2]:
datasetPath='..\kvasir-dataset'
categories=[
    'dyed-lifted-polyps', 
    'dyed-resection-margins',
    'esophagitis',
    'normal-cecum',
    'normal-pylorus',
    'normal-z-line',
    'polyps',
    'ulcerative-colitis'
]
EPOCHS = 25
Learing_Rate = 1e-3
Batch_Size = 32
img_rows = 160
img_cols = 160
img_size = (img_rows,img_cols)

In [3]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    if iteration == total: 
        print()

In [4]:
data = []
labels = []

print("[INFO] loading images...")
imagePaths = sorted(list(paths.list_images(datasetPath)))
random.seed(42)
random.shuffle(imagePaths)

printProgressBar(0, len(imagePaths), prefix = 'Progress:', suffix = 'Complete', length = 50)
for i,imagePath in enumerate(imagePaths):
    image = cv2.imread(imagePath)
    image = cv2.resize(image, img_size)
    image = img_to_array(image)
    data.append(image)
    label = imagePath.split(os.path.sep)[-2]
    label = categories.index(label)
    labels.append(label)
    printProgressBar(i + 1, len(imagePaths), prefix = 'Progress:', suffix = 'Complete', length = 50)

print("[INFO] loading images completed...")

[INFO] loading images...
Progress: |██████████████████████████████████████████████████| 100.0% Complete
[INFO] loading images completed...


In [5]:
data = np.array(data, dtype="float") 
labels = np.array(labels)

(trainX, testX, trainY, testY) = train_test_split(data,labels, test_size=0.25, random_state=42)

trainY = to_categorical(trainY, num_classes=8)
testY = to_categorical(testY, num_classes=8)

In [6]:
print ("number of training examples = " + str(trainX.shape[0]))
print ("number of test examples = " + str(testX.shape[0]))
print ("X_train shape: " + str(trainX.shape))
print ("Y_train shape: " + str(trainY.shape))
print ("X_test shape: " + str(testX.shape))
print ("Y_test shape: " + str(testY.shape))

number of training examples = 3000
number of test examples = 1000
X_train shape: (3000, 160, 160, 3)
Y_train shape: (3000, 8)
X_test shape: (1000, 160, 160, 3)
Y_test shape: (1000, 8)


In [7]:
generator = ImageDataGenerator(
    rotation_range=30, 
    width_shift_range=0.1,
    height_shift_range=0.1, 
    shear_range=0.2, 
    zoom_range=0.2,
    horizontal_flip=True, 
    fill_mode="nearest"
)

In [8]:
def buildModel(width, height, depth, classes):
    inputShape = (height, width, depth)
    if backend.image_data_format() == "channels_first":
        inputShape = (depth, height, width)
    base_model = DenseNet201(
        input_shape=inputShape,
        include_top=False,
        weights='imagenet'
    )
    inputs = tf.keras.layers.Input([height, width, 3], dtype = tf.float32)
    model = tf.keras.applications.densenet.preprocess_input(inputs)
    base_model.trainable = True
    fine_tune_at = int(len(base_model.layers) * 0.64)
    for layer in base_model.layers[:fine_tune_at]:
        layer.trainable = False
    model = base_model(model)
    model = GlobalAveragePooling2D()(model)
    model = Dense(1024,activation='relu')(model)
    model = Dropout(0.2)(model)
    model = Dense(512,activation='relu')(model)
    model = Dense(classes,activation='softmax')(model)
    model = Model(inputs = inputs, outputs = model)
    return model

In [9]:
print("[INFO] compiling model...")
model = buildModel(width=img_cols, height=img_rows, depth=3, classes=8)
model.summary()


[INFO] compiling model...
Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv (TensorF [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd (TensorF [(None, 160, 160, 3)]     0         
_________________________________________________________________
tf_op_layer_RealDiv_1 (Tenso [(None, 160, 160, 3)]     0         
_________________________________________________________________
densenet201 (Functional)     (None, 5, 5, 1920)        18321984  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1920)              0         
_________________________________________________________________
dense (Dense)               

In [12]:
tf.keras.utils.plot_model(
    model,
    show_shapes=False,
    show_layer_names = True,
    to_file="model.png"
)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [10]:
opt = Adam(lr=Learing_Rate, decay=Learing_Rate / EPOCHS)
model.compile(
    loss="categorical_crossentropy", 
    optimizer=opt, 
    metrics=[
        "accuracy",
        tf.keras.metrics.Precision(name="precision"),
        tf.keras.metrics.Recall(name="recall"),
        tf.keras.metrics.SpecificityAtSensitivity(0.5,name="specificity")
    ]
)

In [11]:
print("[INFO] training network...")
history = model.fit(
    x=generator.flow(trainX, trainY, batch_size=Batch_Size),
    validation_data=(testX, testY), 
    steps_per_epoch=len(trainX) // Batch_Size,
    epochs=EPOCHS, 
    verbose=1)
print("[INFO] training network completed...")

[INFO] training network...
Epoch 1/25
93/93 [==============================] - 539s 6s/step - loss: 0.6284 - accuracy: 0.7732 - precision: 0.8137 - recall: 0.7416 - specificity: 0.9948 - val_loss: 0.5046 - val_accuracy: 0.8500 - val_precision: 0.8557 - val_recall: 0.8420 - val_specificity: 0.9979
Epoch 2/25
93/93 [==============================] - 526s 6s/step - loss: 0.3804 - accuracy: 0.8699 - precision: 0.8819 - recall: 0.8558 - specificity: 0.9977 - val_loss: 0.7882 - val_accuracy: 0.8590 - val_precision: 0.8596 - val_recall: 0.8570 - val_specificity: 0.9939
Epoch 3/25
93/93 [==============================] - 545s 6s/step - loss: 0.3082 - accuracy: 0.8986 - precision: 0.9063 - recall: 0.8902 - specificity: 0.9987 - val_loss: 0.3606 - val_accuracy: 0.8930 - val_precision: 0.8956 - val_recall: 0.8920 - val_specificity: 0.9983
Epoch 4/25
93/93 [==============================] - 548s 6s/step - loss: 0.2627 - accuracy: 0.9080 - precision: 0.9129 - recall: 0.9009 - specificity: 0.9992 - 

KeyboardInterrupt: 

In [ ]:
print("[INFO] saving model...")
model.save('Model_DenseNet201-3.model', save_format="h5")
print("[INFO] model saved")

In [ ]:
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Validation Accuracy")
plt.xlabel("Epoch No")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.show()
plt.savefig('plot_DenseNet_201-3.png')

In [ ]:
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_acc")
plt.title("Training and Validation Accuracy")
plt.xlabel("Epoch No")
plt.ylabel("Accuracy")
plt.legend(loc="lower left")
plt.show()
plt.savefig('plot_DenseNet201_Accuracy-3.png')

In [ ]:
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), history.history["val_loss"], label="val_loss")
plt.title("Training and Validation Loss")
plt.xlabel("Epoch No")
plt.ylabel("Loss")
plt.legend(loc="lower left")
plt.show()
plt.savefig('plot_DenseNet201_Loss-3.png')

In [ ]:
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), history.history["precision"], label="train_precision")
plt.plot(np.arange(0, N), history.history["recall"], label="train_recall")
plt.plot(np.arange(0, N), history.history["specificity"], label="train_specificity")
plt.plot(np.arange(0, N), history.history["accuracy"], label="train_accuracy")
plt.plot(np.arange(0, N), history.history["val_precision"], label="val_precision")
plt.plot(np.arange(0, N), history.history["val_recall"], label="val_recall")
plt.plot(np.arange(0, N), history.history["val_specificity"], label="val_specificity")
plt.plot(np.arange(0, N), history.history["val_accuracy"], label="val_accuracy")
plt.title("Training Metrics")
plt.xlabel("Epoch No")
plt.ylabel("Value")
plt.legend(loc="lower left")
plt.show()
plt.savefig('plot_DenseNet201_metrics-3.png')